In [ ]:
%reset

In [ ]:
from os import listdir
from os.path import exists
from importlib import reload
import numpy as np
import pandas as pd
import pyxdf
import mne
from utils import *
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut
import time
import datetime
from datetime import datetime, timezone
import pickle
import plotly.express as px

print('Imports done...')

In [ ]:
#data_path = 'C:/Users/tumfart/Code/github/master-thesis/data/'
data_path = 'C:/Users/peter/Google Drive/measurements/eeg/'
subjects = ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07' , 'A08', 'A09', 'A10']
# = 'A03'
paradigm = 'paradigm' # 'eye', 'paradigm'
plot = False
mne.set_log_level('WARNING')
# Create path list for each subject:
paths = [str(data_path + subject + '/' + paradigm) for subject in subjects]

In [ ]:
# Read all .npy coefficients:
start = time.time()
A = []
for subject, path in zip(subjects, paths):
    file_names = [f for f in listdir(path) if 'regr_coeff.npy' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    A.append(np.load(file))